In [ ]:
# Imports
import torch
from torch import nn
import torch.nn.functional as F
from dotenv import load_dotenv
import wandb
import math
from helpers.memory import check_memory, profile_memory
from helpers.logging import get_gradient_stats
from dataclasses import dataclass, asdict
import time
from collections import defaultdict
import os
import glob 
import json
import dill
from datetime import datetime

main_device = 'cuda:0'
seed = 1234
check_memory()

### Set model configuration settings

In [ ]:
"""
Create a conf with configurable model settings.
- These will be passed into the model class during model initialization, so add new confs needed for whatever architecture is used.
- If you use the default conf values with the default model class defined later, it will exactly replicate the OlMoE-7B model,
   with 7B total params/1B active/64 experts.
"""

@dataclass
class ModelConf:
    """
    General config settings for this MoE
    """
    vocab_size: int = 50304 # Base OlMoE: 50304 (vocab size)
    D: int = 2048 # Base OlMoE: 2048 (hidden state dimension)
    H: int = 16 # Base OlMoE: 16 (number of attention heads)
    I: int = 1024 # Base OlMoE: 1024 (expert MLP dimension)
    n_experts: int = 64 # Base OlMoE: 64 (non-shared experts only)
    n_shared_experts: int = 0 # Base OlMoE: 0 (base OlMoE doesn't support shared experts, but may help with inducing expert specialization - see Deepseek paper)
    top_k: int = 8 # Base OlMoE: 8 
    norm_topk_prob: bool = False # Base OlMoE: false (whether to normalize so that expert weights sum to 1 after topk)
    padding_idx: int = 1 # Base OlMoE: 1 (index where padding gets mapped to)
    n_layers: int = 16 # Base OlMoE: 16 (transformer layers)
    rms_norm_eps: float = 1e-05 # Base OlMoE: 1e-05
    rope_theta: float = 10000.0 # Base OlMoe: 10000.0 (this is something needed for ROPE)
    max_position_embeddings: int = 4096 # Base OlMoE: 4096 (this is something needed for ROPE)
    attn_method: str = 'fa2' # In OlMoE this is chosen automatically, here we explicitly pass it - choose 'normal', 'sdpa', or 'fa2'
    Z: int = 64 # Dimension for the routing residual stream (RRS)    
    rrs_H: int = 1
    
conf = ModelConf(
    D = 768,
    H = 8,
    I = int(640),
    n_layers = 14,
    n_experts = 48,
    n_shared_experts = 1,
    top_k = 4,
    max_position_embeddings = 2048,
    Z = 16,
    rrs_H = 1
)

### Helper funs

In [ ]:
""" 
These is a dump of helper functions called by the model layers, needed to make forward/backward passes correctly.
- `_prepare_4d_causal_attention_mask_with_cache_position` is used to create the upper-triangular infinity mask for attention (not used by flash attention).
- `load_balancing_loss_func` is the usual load balancing function.
- Add any new functions here if needed, but most experiments won't need to touch this section.
"""

# Create the upper-trangular matrix of infinities to mask future tokens in the attention softmax (needed for SDPA + normal attention)
# Copied from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L1099C1-L1152 
def _prepare_4d_causal_attention_mask_with_cache_position(attention_mask: torch.Tensor, sequence_length: int, target_length: int, dtype: torch.dtype, device: torch.device, cache_position: torch.Tensor, batch_size: int):
    if attention_mask is not None and attention_mask.dim() == 4:
        # In this case we assume that the mask comes already in inverted form and requires no inversion or slicing.
        causal_mask = attention_mask
    else:
        min_dtype = torch.finfo(dtype).min
        causal_mask = torch.full((sequence_length, target_length), fill_value = min_dtype, dtype=dtype, device=device)
        if sequence_length != 1:
            causal_mask = torch.triu(causal_mask, diagonal=1)
        causal_mask *= torch.arange(target_length, device=device) > cache_position.reshape(-1, 1)
        causal_mask = causal_mask[None, None, :, :].expand(batch_size, 1, -1, -1)
        if attention_mask is not None:
            causal_mask = causal_mask.clone()  # copy to contiguous memory for in-place edit
            mask_length = attention_mask.shape[-1]
            padding_mask = causal_mask[:, :, :, :mask_length] + attention_mask[:, None, None, :]
            padding_mask = padding_mask == 0
            causal_mask[:, :, :, :mask_length] = causal_mask[:, :, :, :mask_length].masked_fill(
                padding_mask, min_dtype
            )
    return causal_mask

# Load balancing loss, copied from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py
def load_balancing_loss_func(gate_logits, num_experts, top_k, attention_mask):
    compute_device = gate_logits[0].device
    concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim = 0)
    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)
    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
    expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)
    if attention_mask is None:
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)
        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
    else:
        batch_size, sequence_length = attention_mask.shape
        num_hidden_layers = concatenated_gate_logits.shape[0] // (batch_size * sequence_length)
        # Compute the mask that masks all padding tokens as 0 with the same shape of expert_mask
        expert_attention_mask = (attention_mask[None, :, :, None, None].expand((num_hidden_layers, batch_size, sequence_length, top_k, num_experts)).reshape(-1, top_k, num_experts).to(compute_device))
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.sum(expert_mask.float() * expert_attention_mask, dim=0) / torch.sum(expert_attention_mask, dim=0)
        # Compute the mask that masks all padding tokens as 0 with the same shape of tokens_per_expert
        router_per_expert_attention_mask = (attention_mask[None, :, :, None].expand((num_hidden_layers, batch_size, sequence_length, num_experts)).reshape(-1, num_experts).to(compute_device))
        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.sum(routing_weights * router_per_expert_attention_mask, dim=0) / torch.sum(router_per_expert_attention_mask, dim=0)
    overall_loss = torch.sum(tokens_per_expert * router_prob_per_expert.unsqueeze(0))
    return overall_loss * num_experts

### Define model layers

In [ ]:
""" 
First let's define the RMSNorm, ROPE, and self-attention layers.
- These are basically taken straight from the OlMoE source code, but heavily simplified/cleaned up.
- Note that RMSNorm is the ONLY norm type we define (same as OlMoE).
- These layers generally do not need to be modified for MoE experiments.
"""
from transformers.modeling_flash_attention_utils import _flash_attention_forward # Flash attention forward

class OlmoeRMSNorm(nn.Module):
    """
    Apply RMS Norm
    - Copied from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L137-L154
    - This is the only norm used in OlMoE!
      - It's used 4 times per layer (attention key norm, attention query norm, layer residual pre-attention norm, post-attention norm)
      - Also one additional time before the final LM head 
    """
    def __init__(self, D, eps = 1e-5):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(D))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim = True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

class OlmoeRotaryEmbedding(nn.Module):
    """
    Get sin/cos ROPE embeddings
    - Copied from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L161-L219
    - Code has been simplified heavily since we're not using dynamic ROPE scaling
    """
    def __init__(self, conf: ModelConf):
        super().__init__()
        dim = int(conf.D/conf.H)
        inv_freq = 1.0 / (conf.rope_theta ** (torch.arange(0, dim, 2, dtype = torch.int64).float()/dim))
        self.register_buffer("inv_freq", inv_freq, persistent = False)
        
    @torch.no_grad()
    def forward(self, x, position_ids):
        # Core RoPE block
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
        position_ids_expanded = position_ids[:, None, :].float()
        # Force float32 (see https://github.com/huggingface/transformers/pull/29285)
        device_type = x.device.type
        device_type = device_type if isinstance(device_type, str) and device_type != "mps" else "cpu"
        with torch.autocast(device_type = device_type, enabled = False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = torch.cat((freqs, freqs), dim = -1)
            cos = emb.cos()
            sin = emb.sin()
        return cos.to(dtype = x.dtype), sin.to(dtype = x.dtype)

class OlmoeAttention(nn.Module):
    """
    Attention implementation
    - Modified from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L288-L391
    - Simplfied to handle base attention/sdpa/flash attention within this one class
    - Also doesn't support GQA (OlMoE doesn't use anyways)
    """
    def __init__(self, conf: ModelConf):
        super().__init__()
        self.attn_method = conf.attn_method
        self.D = conf.D # Hidden state dim
        self.H = conf.H # Num of attention heads
        self.Dh = int(conf.D/conf.H) # Dimensions per head
        
        # Initialize attention layers - no biases following OlMoE architecture
        self.q_proj = nn.Linear(self.D, self.H * self.Dh, bias = False)
        self.k_proj = nn.Linear(self.D, self.H * self.Dh, bias = False)
        self.v_proj = nn.Linear(self.D, self.H * self.Dh, bias = False)
        self.o_proj = nn.Linear(self.D, self.D, bias = False)
        self.q_norm = OlmoeRMSNorm(self.D, eps = conf.rms_norm_eps)
        self.k_norm = OlmoeRMSNorm(self.D, eps = conf.rms_norm_eps)

    # Taken from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L223-L255
    def apply_rotary_pos_emb(self, q, k, cos, sin, unsqueeze_dim = 1):
        def rotate_half(x):
            """Rotates half the hidden dims of the input."""
            x1 = x[..., : x.shape[-1] // 2]
            x2 = x[..., x.shape[-1] // 2 :]
            return torch.cat((-x2, x1), dim=-1)
            
        cos = cos.unsqueeze(unsqueeze_dim)
        sin = sin.unsqueeze(unsqueeze_dim)
        q_embed = (q * cos) + (rotate_half(q) * sin)
        k_embed = (k * cos) + (rotate_half(k) * sin)
        return q_embed, k_embed

    def forward(self, hidden_state: torch.Tensor, attention_mask: torch.Tensor, position_ids: torch.LongTensor, position_embeddings: tuple[torch.Tensor, torch.Tensor]):
        
        B, N , D = hidden_state.shape

        query_state = self.q_norm(self.q_proj(hidden_state)).view(B, N, self.H, self.Dh).transpose(1, 2) # B x N x 2048
        key_state = self.k_norm(self.k_proj(hidden_state)).view(B, N, self.H, self.Dh).transpose(1, 2) # B x N x 2048
        value_state = self.v_proj(hidden_state).view(B, N, self.H, self.Dh).transpose(1, 2) # B x N x 2048

        cos, sin = position_embeddings
        query_state, key_state = self.apply_rotary_pos_emb(query_state, key_state, cos, sin)
        
        if self.attn_method == 'normal':
            attn_weights = torch.matmul(query_state, key_state.transpose(2, 3))/math.sqrt(self.Dh)  # Should be shape B x H x N x N
            attn_weights = attn_weights + attention_mask # Attention mask is upper triangular of negative infinity
            attn_weights = F.softmax(attn_weights, dim = -1, dtype = torch.float32).to(query_state.dtype)
            attn_output = torch.matmul(attn_weights, value_state) # B x H x N x D/H
            attn_output = attn_output.transpose(1, 2).contiguous() # Reorder into B x N x H x D/H
            attn_output = attn_output.reshape(B, N, D) # Concatenate vertically back into B x N x D
            
        elif self.attn_method == 'sdpa':
            attn_output = torch.nn.functional.scaled_dot_product_attention(
                query_state, key_state, value_state,
                attention_mask, dropout_p = 0.0, is_causal = True
            )
            attn_output = attn_output.transpose(1, 2).contiguous()
            attn_output = attn_output.view(B, N, D)
            
        elif self.attn_method == 'fa2':
            query_state = query_state.transpose(1, 2)
            key_state = key_state.transpose(1, 2)
            value_state = value_state.transpose(1, 2)
            attn_output = _flash_attention_forward(
                query_state, key_state, value_state,
                attention_mask, N, dropout = 0.0, use_top_left_mask = False, is_causal = True
            )
            attn_output = attn_output.reshape(B, N, D).contiguous()
            
        attn_output = self.o_proj(attn_output)
        return attn_output

In [ ]:
""" 
Now let's define the MLP layer and the MoE layer.
- The MLP layer is simple; modify as needed.
- However, the MoE layer is much more complex, and this layer will probably need to be modified heavily for most experiments.
  - By default, I've defined three forward methods here. As currently implemented, they all generate IDENTICAL outputs but become increasingly more efficient yet complex.
    - `forward_slow` is the most straightforward implementation (similar to the original OlMoE code). It is also the fastest for single-GPU, limited experts (32 or less) operations.
    - `forward_fast` is faster for large # experts, as it places all the relevant states for a single expert to be continguous in memory. For single GPU, it reaches parity w/forward_slow at ~64 experts.
    - `forward_async` is faster for large GPU counts + large # of experts, as it batches all experts who belong on one device together, and also runs them all asynchronously.
    - For initial testing, it's probably best to modify just `forward_slow`, and only modify the others once you want to run a large-scale training run.
  - Each forward method must return a tuple where the first element is the B x N x D MoE layer output, and the second element is the router logits. 
    - To return more, you'll need to also modify the transformer layer class in the next section.
"""
from transformers.activations import silu

class OlmoeMLP(nn.Module):
    """
    Individual expert MLP
    - Copied from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L258-L272
    """
    def __init__(self, conf: ModelConf):
        super().__init__()
        self.conf = conf
        self.D = conf.D
        self.I = conf.I
        self.gate_proj = nn.Linear(self.D, self.I, bias = False)
        self.up_proj = nn.Linear(self.D, self.I, bias = False)
        self.down_proj = nn.Linear(self.I, self.D, bias = False)
        self.act_fn = silu

    def forward(self, x):
        down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
        return down_proj

class OlmoeMoe(nn.Module):
    """
    Entire MLP layer including router
    - Modified from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L604-L649
    """
    def __init__(self, conf: ModelConf, shared_gate: nn.Linear):
        super().__init__()
        self.n_experts = conf.n_experts
        self.top_k = conf.top_k
        self.norm_topk_prob = conf.norm_topk_prob
        self.n_shared_experts = conf.n_shared_experts

        # The globally shared gating (created in top-level model)
        self.shared_gate = shared_gate  # Not newly created here, an inherited reference
        self.gate_offset = nn.Linear(conf.Z, self.n_experts, bias = False) # Per-layer offset gating (this is what will be penalized)

        self.experts = nn.ModuleList([OlmoeMLP(conf) for _ in range(self.n_experts)]) # Create experts using OlmoeMLP
        self.shared_experts = nn.ModuleList([OlmoeMLP(conf) for _ in range(self.n_shared_experts)])
        
        # Loss-free load balancing bias ----------------
        # We store a bias value b_i for each expert i.  These are NOT updated by backprop.
        self.register_buffer("expert_biases", torch.zeros(self.n_experts))

    def forward(self, hidden_state: torch.Tensor, rrs_state: torch.Tensor, moe_method: str, use_lflb: bool = False) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Forward method routes to one of several possible other forward methods
        """
        if moe_method == 'forward_slow':
            moe_output, router_logits, topk_expert_ids = self.forward_slow(hidden_state, rrs_state, use_lflb)
        elif moe_method == 'forward_fast':
            moe_output, router_logits, topk_expert_ids = self.forward_fast(hidden_state, rrs_state, use_lflb)
        elif moe_method == 'forward_async':
            moe_output, router_logits, topk_expert_ids = self.forward_async(hidden_state, rrs_state, use_lflb)
        else:
            raise ValueError(f'Method "{moe_method}" not implemented.')
        
        # Add output from shared experts
        shared_total = torch.zeros_like(moe_output)
        for shared_expert in self.shared_experts:
            shared_total += shared_expert(hidden_state)

        mlp_output = moe_output + shared_total
        return mlp_output, router_logits, topk_expert_ids
        
    # -------------------- LOSS-FREE BIAS UPDATE FUNCTION --------------------
    @torch.no_grad()
    def update_expert_biases(self, usage_counts: dict[int, int], bias_update_rate: float) -> None:
        """
        Given a dict mapping expert_id -> usage_count for this batch, push biases up/down to move usage closer to uniform.
        """
        total_tokens = sum(usage_counts.values())
        mean_usage = total_tokens / self.n_experts

        for ex_id in range(self.n_experts):
            ex_count = usage_counts.get(ex_id, 0) # usage_counts might not have an entry for an expert if it got 0 tokens
            diff = mean_usage - ex_count # e_i = mean_usage - ex_count

            # Sign-based udpate
            if diff > 0:
                self.expert_biases[ex_id] += bias_update_rate
            elif diff < 0:
                self.expert_biases[ex_id] -= bias_update_rate
                
        # Optionally: clamp the biases to avoid extreme values
        self.expert_biases.clamp_(-5.0, 5.0)
                
    def forward_slow(self, hidden_state: torch.Tensor, rrs_state: torch.Tensor, use_lflb: bool) -> tuple[torch.Tensor, torch.Tensor]:
        """
        This is the more intuitive forward pass which loops through each expert slowly
        """
        B, N, D = hidden_state.shape

        # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        hidden_state = hidden_state.view(B * N, D) # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        rrs_state = rrs_state.view(B * N, -1) # BN x Z

        # 1) ---------------- Compute router logits -> choose topk experts and weights ----------------
        router_logits = self.shared_gate(rrs_state) + self.gate_offset(rrs_state) # (BN, n_experts) - routing probability for each token
        routing_weights = F.softmax(router_logits, dim = 1, dtype = torch.float)  # (BN, n_experts)

        # topk_weights and topk_expert_ids  represents for each token, the top k experts and the corresponding weights/expert indices
        if use_lflb:
            router_logits = router_logits + self.expert_biases.unsqueeze(0)
            _, topk_expert_ids  = torch.topk(router_logits, self.top_k, dim = -1) # (BN, top_k)
            topk_weights = torch.gather(routing_weights, 1, topk_expert_ids ) # (BN, top_k)
        else:
            topk_weights, topk_expert_ids  = torch.topk(routing_weights, self.top_k, dim = -1) # both (BN, top_k)

        topk_weights = topk_weights.to(hidden_state.dtype)
        # Optional renormalization if you want the top-k weights to sum to 1
        if self.norm_topk_prob:
            topk_weights /= (topk_weights.sum(dim = -1, keepdim = True) + 1e-9)

        # 2) ---------------- One hot encode - for each expert, which topk x token is active - e.g. expert_assignment_mask[0, :] will be 0s if the first expert is never chosen ----------------
        expert_assignment_mask = F.one_hot(topk_expert_ids , num_classes = self.n_experts).permute(2, 1, 0) # Creates (N_EXPERTS, TOP_K, BN)
        
        # 3) ---------------- Iterate through all the experts, apply each expert to the tokens where the expert are relevant, multiple output by the weights for the topk/token for that expert ----------------
        # Initialize output buffer
        mlp_output = torch.zeros((B * N, D), dtype = hidden_state.dtype, device = hidden_state.device) # Initialize MLP output - later iterate through experts and sum onto this object
        # Iterate
        for expert_ix, expert in enumerate(self.experts):
            
            # For this expert, gives the (topk, token) coordinates which uses the expert
            topk_slot, token_indices = torch.where(expert_assignment_mask[expert_ix, :])
            if token_indices.numel() == 0:
                continue

            # Gather input tokens for this expert
            tokens_for_expert = hidden_state[token_indices, :] # (num_assigned_tokens, D)
            
            # Move to the expert's device
            expert_device = next(self.experts[expert_ix].parameters()).device # Get the device this expert lives on
            tokens_for_expert = tokens_for_expert.to(expert_device) # Move to expert device

            # Forward through expert
            expert_output = expert(tokens_for_expert)
          
            # For each num_assigned_tokens, multiples it by the corresponding weight in topk_slot fort that token_index
            expert_output = expert_output * topk_weights[token_indices, topk_slot].unsqueeze(1).to(expert_device)

            # Move back to original device and acucmulate into mlp output
            expert_output = expert_output.to(mlp_output.device)
            mlp_output.index_add_(0, token_indices, expert_output.to(hidden_state.dtype))

        mlp_output = mlp_output.reshape(B, N, D) # Convert back from BN x D -> B x N x D
        return mlp_output, router_logits, topk_expert_ids
    

    def forward_fast(self, hidden_state: torch.Tensor, rrs_state: torch.Tensor, use_lflb: bool) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Efficient MoE routing that batches tokens for each expert in a single pass using gather -> scatter operations. 
        - This will be much faster for a large number of experts, but possibly lower for low expert counts.
        """
        B, N, D = hidden_state.shape

        # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        hidden_state = hidden_state.view(B * N, D) # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        rrs_state = rrs_state.view(B * N, -1) # BN x Z

        # 1) ---------------- Compute router logits -> choose topk experts and weights ----------------
        router_logits = self.shared_gate(rrs_state) + self.gate_offset(rrs_state) # (BN, n_experts) - routing probability for each token
        routing_weights = F.softmax(router_logits, dim = 1, dtype = torch.float)  # (BN, n_experts)

        # topk_weights and topk_expert_ids  represents for each token, the top k experts and the corresponding weights/expert indices
        if use_lflb:
            router_logits = router_logits + self.expert_biases.unsqueeze(0)
            _, topk_expert_ids = torch.topk(router_logits, self.top_k, dim = -1) # (BN, top_k)
            topk_weights = torch.gather(routing_weights, 1, topk_expert_ids) # (BN, top_k)
        else:
            topk_weights, topk_expert_ids = torch.topk(routing_weights, self.top_k, dim = -1) # both (BN, top_k)

        # 2) ---------------- Flatten topk results so we can later gather tokens for each expert cleanly ----------------
        topk_expert_ids = topk_expert_ids.view(-1)
        topk_weights = topk_weights.view(-1)

        # [0..BN-1], repeated top_k times (one for each token-expert pair)
        token_indices_flat = torch.arange(B * N, device = hidden_state.device).unsqueeze(1).expand(B * N, self.top_k).reshape(-1) # shape = (BN * top_k,)

        # 3) ---------------- Sort by expert index so we can process contiguous tokens for each expert ----------------
        sorted_expert_ids, expert_sort_indices = torch.sort(topk_expert_ids)
        sorted_token_indices = token_indices_flat[expert_sort_indices]
        sorted_weights = topk_weights[expert_sort_indices]
        sorted_inputs = hidden_state[sorted_token_indices]  # shape = (BN*top_k, D)

        # 4) ---------------- Walk through sorted_experts to find contiguous segments belonging to each expert. We can use torch.unique_consecutive to figure out segment boundaries ----------------
        # Initialize output buffer
        mlp_output = torch.zeros((B * N, D), dtype = hidden_state.dtype, device = hidden_state.device) # Initialize MLP output - later iterate through experts and sum onto this object
        unique_expert_ids, counts = torch.unique_consecutive(sorted_expert_ids, return_counts = True)
        # Iterate
        start_offset = 0
        for expert_id, count_for_this_expert in zip(unique_expert_ids, counts):
            end_offset = start_offset + count_for_this_expert # The chunk [start_offset : end_offset] corresponds to all tokens for this expert

            # Indices for this chunk
            chunk_token_indices = sorted_token_indices[start_offset:end_offset] # (count, D)
            chunk_weights = sorted_weights[start_offset:end_offset].unsqueeze(1) # (count, 1)
            chunk_inputs = sorted_inputs[start_offset:end_offset] # (count, )

            start_offset = end_offset

            # Move to device
            expert_device = next(self.experts[expert_id].parameters()).device
            chunk_inputs = chunk_inputs.to(expert_device)
            chunk_weights = chunk_weights.to(expert_device)

            # Forward pass through this expert
            expert = self.experts[expert_id]
            expert_output = expert(chunk_inputs) # (count, D)

            # Multiply by the top-k gate weight
            expert_output = expert_output * chunk_weights

            # Bring it back to the main device and scatter-add back to the correct token positions
            expert_output = expert_output.to(mlp_output.device)
            mlp_output.index_add_(0, chunk_token_indices, expert_output.to(hidden_state.dtype))

        mlp_output = mlp_output.reshape(B, N, D) # Convert back from BN x D -> B x N x D
        return mlp_output, router_logits, topk_expert_ids
    

    def forward_async(self, hidden_state: torch.Tensor, rrs_state: torch.Tensor, use_lflb: bool) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Async MoE forward for optimal multi-GPU speeds that:
          1) Flattens tokens
          2) Does gating + top-k
          3) Sorts by (device, expert)
          4) Groups tokens by device, then sub-groups by expert
          5) Uses asynchronous CUDA streams to overlap transfers & expert compute
          6) Accumulates outputs back to the main device
        This is ~2.5x faster than forward_slow with 64 experts on 4 GPUs
        """
        B, N, D = hidden_state.shape        
        main_device = hidden_state.device
        expert_device_map = [str(next(expert.parameters()).device) for expert in self.experts]

        # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        hidden_state = hidden_state.view(B * N, D) # Flatten out B x N x D to BN x D (flattened token-level reps) to route all tokens seperately
        rrs_state = rrs_state.view(B * N, -1) # BN x Z
        
        # ---------------- 1) Compute router logits -> choose topk experts and weights ----------------
        router_logits = self.shared_gate(rrs_state) + self.gate_offset(rrs_state) # (BN, n_experts) - routing probability for each token
        routing_weights = F.softmax(router_logits, dim = 1, dtype = torch.float)  # (BN, n_experts)

        # topk_weights and topk_expert_ids  represents for each token, the top k experts and the corresponding weights/expert indices
        if use_lflb:
            router_logits = router_logits + self.expert_biases.unsqueeze(0)
            _, topk_expert_ids = torch.topk(router_logits, self.top_k, dim = -1) # (BN, top_k)
            topk_weights = torch.gather(routing_weights, 1, topk_expert_ids) # (BN, top_k)
        else:
            topk_weights, topk_expert_ids = torch.topk(routing_weights, self.top_k, dim = -1) # both (BN, top_k)

        # ---------------- 2) Flatten topk results so we can later gather tokens for each expert cleanly ----------------
        topk_expert_ids = topk_expert_ids.view(-1)
        topk_weights = topk_weights.view(-1)

        # [0..BN-1], repeated top_k times (one for each token-expert pair)
        token_indices_flat = torch.arange(B * N, device = hidden_state.device).unsqueeze(1).expand(B * N, self.top_k).reshape(-1) # shape = (BN * top_k,)

        # ---------------- 3) Sort by expert index so we can process contiguous tokens for each expert ----------------
        sorted_expert_ids, expert_sort_indices = torch.sort(topk_expert_ids)
        sorted_token_indices = token_indices_flat[expert_sort_indices]
        sorted_weights = topk_weights[expert_sort_indices]
        sorted_inputs = hidden_state[sorted_token_indices]  # shape = (BN*top_k, D)

        # ---------------- 4) Build a dict of (expert -> list of tokens) grouped by device  ---------------
        device_to_chunk = defaultdict(list)

        for i in range(sorted_expert_ids.size(0)):
            ex_id = sorted_expert_ids[i].item() # Which expert
            tok_id = sorted_token_indices[i].item() # The actual token index
            w_val = sorted_weights[i].item() # Probability for that token-expert pair
            inp_vec = sorted_inputs[i] # The input embedding
            ex_dev = expert_device_map[ex_id]  # for example, 'cuda:1'
            device_to_chunk[ex_dev].append((ex_id, tok_id, w_val, inp_vec)) # We store enough info to re-group by expert on the same device

        # ---------------- 5) Create streams for each device used ---------------
        device_streams = {}
        for dev_str in set(expert_device_map):
            device_streams[dev_str] = torch.cuda.Stream(device = dev_str)
        # Store partial outputs in a dict of device -> list of (token_ids, output_tensors)
        device_results = defaultdict(list)

        # ---------------- 6) Dispatch to each device in its stream ---------------
        # We'll do a loop over the devices. For each device, we do sub-grouping by expert ID
        for dev_str, tuple_list in device_to_chunk.items():
            if len(tuple_list) == 0:
                continue

            with torch.cuda.stream(device_streams[dev_str]):
                tuple_list.sort(key=lambda x: x[0])  # Sort by ex_id : each tuple is (expert_id, token_idx, w_val, inp_vec)

                # Sub-group by expert
                idx_start = 0
                while idx_start < len(tuple_list):
                    current_ex = tuple_list[idx_start][0]
                    idx_end = idx_start
                    # Gather all entries with the same expert_id
                    while idx_end < len(tuple_list) and tuple_list[idx_end][0] == current_ex:
                        idx_end += 1
                    sub_chunk = tuple_list[idx_start:idx_end]
                    idx_start = idx_end

                    # Unzip the sub-chunk
                    token_ids = [sc[1] for sc in sub_chunk]
                    w_vals = [sc[2] for sc in sub_chunk]
                    inps = [sc[3] for sc in sub_chunk]

                    # Move to main_device -> dev_str as needed
                    token_ids_t = torch.tensor(token_ids, device = main_device, dtype = torch.long)
                    w_vals_t = torch.tensor(w_vals, device = main_device, dtype = hidden_state.dtype)
                    inps_t = torch.stack(inps, dim=0).to(dev_str, non_blocking = True)
                    w_vals_t = w_vals_t.unsqueeze(1).to(dev_str, non_blocking = True)

                    # Forward pass on the expert (on expert device)
                    chunk_output = self.experts[current_ex](inps_t)
                    # Multiply by gating weights
                    chunk_output = chunk_output * w_vals_t  # gating
                    # Move output back to main device
                    chunk_output = chunk_output.to(main_device, non_blocking=True)
                    # Store for now, and index_add_ after we sync
                    device_results[dev_str].append((token_ids_t, chunk_output))

        # ---------------- 7) Synchronize & gather on main device ----------------
        mlp_output = torch.zeros_like(hidden_state, dtype = hidden_state.dtype)

        for dev_str, stream in device_streams.items():
            # Wait for everything launched in that stream to finish
            with torch.cuda.device(dev_str):
                stream.synchronize()

            # Now we can safely do index_add_ on main_device
            for (tok_ids, out_vecs) in device_results[dev_str]:
                mlp_output.index_add_(0, tok_ids, out_vecs)

        # Reshape to [B, N, D]
        mlp_output = mlp_output.view(B, N, D)
        return mlp_output, router_logits, topk_expert_ids


In [ ]:
""" 
Now let's define the transformer block.
- Most likely, there is nothing to change here, unless you need to change the input/outputs from the MoE layer.
- Note that this forward pass is nested within a `custom_forward` call in order to support gradient checkpointing.
"""

class Downcast(nn.Module):
    def __init__(self, D: int, Z: int):
        super().__init__()
        self.down_lin = nn.Linear(D, Z, bias = False)
        self.down_gate = nn.Linear(D, Z, bias = False)
        self.down_act = silu

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.down_lin(x) * self.down_act(self.down_gate(x))
    
class RRSRotaryEmbedding(nn.Module):
    def __init__(self, conf: ModelConf):
        super().__init__()
        dim = int(conf.Z/conf.rrs_H)
        inv_freq = 1.0 / (conf.rope_theta ** (torch.arange(0, dim, 2, dtype = torch.int64).float()/dim))
        self.register_buffer("inv_freq", inv_freq, persistent = False)
        
    @torch.no_grad()
    def forward(self, x, position_ids):
        # Core RoPE block
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
        position_ids_expanded = position_ids[:, None, :].float()
        # Force float32 (see https://github.com/huggingface/transformers/pull/29285)
        device_type = x.device.type
        device_type = device_type if isinstance(device_type, str) and device_type != "mps" else "cpu"
        with torch.autocast(device_type = device_type, enabled = False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = torch.cat((freqs, freqs), dim = -1)
            cos = emb.cos()
            sin = emb.sin()
        return cos.to(dtype = x.dtype), sin.to(dtype = x.dtype)

class RRSAttention(nn.Module):
    """
    A mini self-attention mechanism for the RRS, 
    parallel to the standard OlmoeAttention approach but specialized for dimension Z.
    """
    def __init__(self, conf: ModelConf):
        super().__init__()
        self.D = conf.Z  # The dimension of the RRS
        self.rrs_H = conf.rrs_H # or any default you like
        self.head_dim = self.D // self.rrs_H

        # Optional RMS norms on Q/K
        self.q_norm = OlmoeRMSNorm(self.D, eps=conf.rms_norm_eps)
        self.k_norm = OlmoeRMSNorm(self.D, eps=conf.rms_norm_eps)

        # Linear projections (no bias, matching your style)
        self.q_proj = nn.Linear(self.D, self.rrs_H * self.head_dim, bias=False)
        self.k_proj = nn.Linear(self.D, self.rrs_H * self.head_dim, bias=False)
        self.v_proj = nn.Linear(self.D, self.rrs_H * self.head_dim, bias=False)
        self.o_proj = nn.Linear(self.D, self.D, bias=False)

    # Taken from https://github.com/huggingface/transformers/blob/main/src/transformers/models/olmoe/modeling_olmoe.py#L223-L255
    def apply_rotary_pos_emb(self, q, k, cos, sin, unsqueeze_dim = 1):
        def rotate_half(x):
            """Rotates half the hidden dims of the input."""
            x1 = x[..., : x.shape[-1] // 2]
            x2 = x[..., x.shape[-1] // 2 :]
            return torch.cat((-x2, x1), dim=-1)
            
        cos = cos.unsqueeze(unsqueeze_dim)
        sin = sin.unsqueeze(unsqueeze_dim)
        q_embed = (q * cos) + (rotate_half(q) * sin)
        k_embed = (k * cos) + (rotate_half(k) * sin)
        return q_embed, k_embed

    def forward(
        self,
        rrs_state: torch.Tensor,       # (B, N, Z)
        attention_mask: torch.Tensor,  # (B, 1, N, N) or (B, N, N) - same shape as normal attn
        position_ids: torch.LongTensor,
        position_embeddings: tuple[torch.Tensor, torch.Tensor]
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Returns:
          rrs_attn_output: (B, N, Z)
          nonself_attn_score: scalar float measuring how much attention each token devotes to tokens != itself
        """
        B, N, Z = rrs_state.shape

        # 1) Project Q/K/V from rrs_state, with optional RMS norm on Q/K
        query_state = self.q_proj(self.q_norm(rrs_state)).view(B, N, self.rrs_H, self.head_dim).transpose(1, 2)
        key_state = self.k_proj(self.k_norm(rrs_state)).view(B, N, self.rrs_H, self.head_dim).transpose(1, 2)
        value_state = self.v_proj(rrs_state).view(B, N, self.rrs_H, self.head_dim).transpose(1, 2)

        cos, sin = position_embeddings
        query_state, key_state = self.apply_rotary_pos_emb(query_state, key_state, cos, sin)

        attn_weights = torch.matmul(query_state, key_state.transpose(2, 3))/math.sqrt(self.head_dim)  # Should be shape B x H x N x N
        attn_weights = attn_weights + attention_mask # Attention mask is upper triangular of negative infinity
        attn_weights = F.softmax(attn_weights, dim = -1, dtype = torch.float32)
        attn_output = torch.matmul(attn_weights.to(query_state.dtype), value_state) # B x H x N x D/H
        attn_output = attn_output.transpose(1, 2).contiguous() # Reorder into B x N x H x D/H
        attn_output = attn_output.reshape(B, N, Z) # Concatenate vertically back into B x N x D

        # 6) "nonself_attn_score" calculation: 
        #    For each token i, fraction of attention NOT on i itself => 1 - attn_weights[..., i, i].
        #    Then average over all tokens + heads + batch.
        #    shape of diag: (B, H, N)
        # print(attn_weights.shape)
        diag_weights = attn_weights[..., range(N), range(N)]  # gather diagonal
        # print('diag_weights')
        # print(diag_weights)
        # print(diag_weights.shape)
        nonself_per_token = 1.0 - diag_weights  # fraction on others
        # Avg out H/N then B
        nonself_attn_score = nonself_per_token.mean(dim = [1, 2]).mean()  # scalar
        
        rrs_attn_output = self.o_proj(attn_output)

        return rrs_attn_output, nonself_attn_score

class OlmoeBlock(nn.Module):
    """
    A single transformer layer
    """
    def __init__(self, conf: ModelConf, layer_idx: int, shared_gate: nn.Linear):
        super().__init__()
        self.D = conf.D
        self.self_attn = OlmoeAttention(conf = conf)
        # self.self_attn = torch.compile(self.self_attn) # attn layers can be compiled for speed 
        self.moe = OlmoeMoe(conf, shared_gate = shared_gate)
        self.input_layernorm = OlmoeRMSNorm(conf.D, eps = conf.rms_norm_eps)
        self.post_attention_layernorm = OlmoeRMSNorm(conf.D, eps = conf.rms_norm_eps)

        # Downcast from D->Z for RRS accumulation
        # self.downcast = nn.Linear(conf.D, conf.Z, bias = False)
        # self.downcast = nn.Sequential(
        #     nn.Linear(conf.D, conf.Z * 2, bias = False),
        #     nn.ReLU(),
        #     nn.Linear(conf.Z * 2, conf.Z, bias = False),
        #     OlmoeRMSNorm(conf.Z, eps = conf.rms_norm_eps)  # or RMSNorm
        # )
        self.downcast = Downcast(conf.D, conf.Z)
        self.rrs_norm = OlmoeRMSNorm(conf.Z, eps=conf.rms_norm_eps)
        
        # Alpha param: 0.0 -> .50 after 0.5 + 0.5 * tanh(alpha)
        self._alpha_param = nn.Parameter(torch.tensor(0.0, dtype=torch.float32))

        # Gamma param: 0.0 -> .50 after 0.5 + 0.5 * tanh(alpha)
        self.rrs_attn = RRSAttention(conf)  # our new module
        self._gamma_param = nn.Parameter(torch.tensor(0.0, dtype=torch.float32))

    def _block_forward(
        self,
        hidden_state: torch.Tensor,
        rrs_state: torch.Tensor,
        attention_mask: torch.Tensor,
        rrs_mask: torch.Tensor,
        position_ids: torch.LongTensor,
        position_embeddings: tuple[torch.Tensor, torch.Tensor],
        rrs_position_embeddings: tuple[torch.Tensor, torch.Tensor],
        moe_method: str,
        use_lflb: bool = False
        ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:

        ### Pre-SA Residual Stream + Norm ###
        residual = hidden_state
        hidden_state = self.input_layernorm(hidden_state)
        
        ### SA + Sum to Residual Stream ###
        attn_output = self.self_attn(
            hidden_state,
            attention_mask = attention_mask,
            position_ids = position_ids,
            position_embeddings = position_embeddings
        )
        hidden_state = residual + attn_output

        ### Pre-MLP Residual Stream + Norm ###
        residual = hidden_state
        hidden_state = self.post_attention_layernorm(hidden_state)
        
        downcast_proj = self.downcast(hidden_state)  # B x N x Z

        alpha = 0.5 + 0.5 * torch.tanh(self._alpha_param)  # scalar, alpha in [0, 1]

        rrs_state = (1 - alpha) * rrs_state + alpha.to(rrs_state.dtype) * downcast_proj # Sum to residual stream
        rrs_state = self.rrs_norm(rrs_state)

        # ---- Window aggregator for RRS ----
        rrs_attn_output, nonself_attn_score = self.rrs_attn(
            rrs_state,
            attention_mask = rrs_mask,
            position_ids = position_ids,
            position_embeddings = rrs_position_embeddings
        )
        # V1
        rrs_attn_output = self.rrs_norm(rrs_attn_output + rrs_state)
        # V2
        # gamma = 0.5 + 0.5 * torch.tanh(self._gamma_param)
        # rrs_dtype = rrs_state.dtype
        # rrs_state = ((1 - gamma) * rrs_state + (gamma * rrs_attn_output)).to(rrs_dtype)

        ### MLP + Sum to Residual Stream###
        mlp_output, router_logits, topk_experts = self.moe(hidden_state = hidden_state, rrs_state = rrs_state, moe_method = moe_method, use_lflb = use_lflb)
        hidden_state = residual + mlp_output
        
        return hidden_state, rrs_state, router_logits, topk_experts, nonself_attn_score
        
    def forward(
        self,
        hidden_state: torch.Tensor,
        rrs_state: torch.Tensor,
        attention_mask: torch.Tensor,
        rrs_mask: torch.Tensor,
        position_ids: torch.LongTensor,
        position_embeddings: tuple[torch.Tensor, torch.Tensor],
        rrs_position_embeddings: tuple[torch.Tensor, torch.Tensor],
        moe_method: str,
        use_lflb: bool,
        use_checkpointing: bool
        ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
            
        if use_checkpointing:
            return torch.utils.checkpoint.checkpoint(
                self._block_forward,
                hidden_state, rrs_state, attention_mask, rrs_mask, position_ids,
                position_embeddings, rrs_position_embeddings, moe_method, use_lflb,
                use_reentrant = True
            )
        else:
            return self._block_forward(
                hidden_state, rrs_state, attention_mask, rrs_mask, position_ids,
                position_embeddings, rrs_position_embeddings, moe_method, use_lflb
            )
    
    def get_rrs_offset_loss(self) -> torch.Tensor:
        """
        Return the L2 norm of the gating offset for this layer's MoE, so the top-level model can sum them up.
        """
        return (self.moe.gate_offset.weight ** 2).mean()

In [ ]:
""" 
Now define the top-level model.
- This class is initialized with the `ModelConf` config settings as well as a list of expert-device mappings (leave blank for single-GPU tests).
- After initialization, it creates all child layers and moves the experts to their correct devices. 
  - All other parameters will continue to exist on the default device.
- Modify `_init_weights` to change the weight initialization scheme.
- The forward pass calls the children layers and also calculates the loss (standard cross-entropy + aux loss). 
"""
from transformers.loss.loss_utils import ForCausalLMLoss # Cross-entropy loss that handles label shifting

class OlmoeModel(nn.Module):
    """
    The top level model object. Also handles weight initialization and loss calculations.
    """
    
    def __init__(self, conf: ModelConf, primary_device: str, expert_device_map: None|list[str] = None):
        """
        Params:
            @conf: A configuration object of class ModelConf.
            @primary_device: A device for which to store the dense layers and shared experts on.
            @expert_device_map: A list of devices to store experts on. If `None`, stores them all on whatever the torch default device is.
              For example, `expert_device_map = ['cuda:0', 'cuda:1', 'cuda:1', 'cuda:2']` means to store expert 0 on cuda:0, experts 1-2 on the device cuda:1, and expert 3 on cuda:2.
        """
        super().__init__()
        self.conf = conf
        
        ### Layers ###
        self.combined_embedding = nn.Embedding(self.conf.vocab_size, self.conf.D + self.conf.Z, self.conf.padding_idx)
        self.rotary_emb = OlmoeRotaryEmbedding(conf = self.conf)
        self.rrs_rotary_emb = RRSRotaryEmbedding(conf = self.conf)

        self.shared_gate = nn.Linear(conf.Z, conf.n_experts, bias=False)

        self.layers = nn.ModuleList([OlmoeBlock(self.conf, layer_idx, shared_gate = self.shared_gate) for layer_idx in range(self.conf.n_layers)])
        self.norm = OlmoeRMSNorm(self.conf.D, eps = self.conf.rms_norm_eps)
        self.lm_head = nn.Linear(self.conf.D, self.conf.vocab_size, bias = False)
        
        ### Initialize weights ###
        self.apply(self._init_weights)

        ### Model ###
        self.to(primary_device)

        ### Experts ###
        if expert_device_map is not None:
            self._move_experts_to_devices(expert_device_map)

    # OlMoE weight initiation - see https://github.com/huggingface/transformers/blob/8f1509a96c96747c893051ac947795cfb0750357/src/transformers/modeling_utils.py#L2500-L2515
    # Normal distribution for linear layers + embeddings
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean = 0.0, std = 0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean = 0.0, std = 0.02)
            # In the vocab -> embedding layer, set all embeddings to 0 for the padding token (tokenizer.pad_token_id)
            if module is self.combined_embedding:
                module.weight.data[self.conf.padding_idx].zero_() 
            
    def _move_experts_to_devices(self, expert_device_map: list[str]):
        """
        Move each expert in each layer's MoE to the specified device.
        """
        # Require that the length of expert_device_map equal the length of conf.n_experts.
        n_experts = self.conf.n_experts
        if len(expert_device_map) != n_experts:
            raise ValueError(f"expert_device_map has length {len(expert_device_map)} but n_experts = {n_experts}.")
            
        for _, layer in enumerate(self.layers):
            moe_block = layer.moe 
            for ex_idx, expert in enumerate(moe_block.experts):
                target_dev = expert_device_map[ex_idx]
                expert.to(target_dev)
            
    def forward(self, input_ids: torch.LongTensor, attention_mask: torch.Tensor, moe_method: str, use_lflb: bool = False, use_checkpointing : bool = False):
        """
        Params:
            @input_ids: A tensor of input IDs of size B x N, where B is the batch size and N is the sequence length.
            @attention_mask: An attention mask tensor of size B x N.
            @moe_method: The method to use to calculate the MoE routing. See the `OlmoeMoe` class for details.
            @use_lflb: Whether or not to use loss-free balancing.
            @use_checkpointing: Whether to use gradient checkpointing. Only set `True` during training.
        """
        combined_state = self.combined_embedding(input_ids)  # (B, N, D+Z)
        hidden_state = combined_state[..., :self.conf.D]
        rrs_state = combined_state[..., self.conf.D:]
        
        B, N, D = hidden_state.shape

        ### Prep rotary embeddings + attention masks  ###
        cache_position = torch.arange(0, N, device = hidden_state.device)
        position_ids = cache_position.unsqueeze(0)
        position_embeddings = self.rotary_emb(hidden_state, position_ids) # Position embeddings to be shared across transformer layers

        # This is the upper-trangular matrix of infinities to mask future tokens in the attention softmax;
        if self.conf.attn_method in ['normal', 'sdpa']:
            causal_mask = _prepare_4d_causal_attention_mask_with_cache_position(attention_mask, N, N, hidden_state.dtype, hidden_state.device, cache_position, B)
        # The flash attention mask is simpler - takes only the original attention mask or None
        elif self.conf.attn_method == 'fa2':
            causal_mask = attention_mask if (attention_mask is not None and torch.any(attention_mask == 0)) else None

        rrs_mask = _prepare_4d_causal_attention_mask_with_cache_position(attention_mask, N, N, hidden_state.dtype, hidden_state.device, cache_position, B)
        rrs_position_embeddings = self.rrs_rotary_emb(rrs_state, position_ids) # Position embeddings to be shared across transformer layers
        
        ### Transformer layers ###
        all_router_logits = () # Save router logits from each layer into this; will be needed for load balancing loss
        all_topk_experts = () # Return topk experts
        all_rrs_states = () # Collect rrs states for metrics
        all_nonself_scores = ()

        # We'll accumulate offset_loss across layers
        total_rrs_offset_loss = torch.zeros((), device = hidden_state.device)

        for _, layer in enumerate(self.layers):
            hidden_state, rrs_state, router_logits, topk_experts, nonself_attn_score = layer(
                hidden_state,
                rrs_state = rrs_state,
                attention_mask = causal_mask,
                rrs_mask = rrs_mask,
                position_ids = position_ids,
                position_embeddings = position_embeddings,
                rrs_position_embeddings = rrs_position_embeddings,
                moe_method = moe_method,
                use_lflb = use_lflb,
                use_checkpointing = use_checkpointing
            )
            all_router_logits += (router_logits, )
            all_topk_experts += (topk_experts,)  # Store the topk_experts for each layer

            all_rrs_states += (rrs_state.clone(),)
            total_rrs_offset_loss += layer.get_rrs_offset_loss() # Accumulate RRS offset penalty from this layer
            all_nonself_scores += (nonself_attn_score,)

        hidden_state = self.norm(hidden_state)
        output_logits = self.lm_head(hidden_state)

        ##### Calculate Loss #####
        # The labels object should be a tensor of token IDs or -100 (for attention mask, since don't want to calculate loss for those)
        label_ids = torch.where(input_ids == self.conf.padding_idx, torch.tensor(-100), input_ids)
        # Get regular loss
        base_loss = ForCausalLMLoss(output_logits, label_ids, self.conf.vocab_size)
        # Get load balancing loss
        aux_loss = load_balancing_loss_func(gate_logits = all_router_logits, num_experts = self.conf.n_experts, top_k = self.conf.top_k, attention_mask = attention_mask)
        
        return {
            'all_router_logits': all_router_logits,
            'all_topk_experts': all_topk_experts,
            'all_rrs_states': all_rrs_states,
            'logits': output_logits,
            'aux_loss': aux_loss,
            'base_loss': base_loss,
            'rrs_offset_loss': total_rrs_offset_loss,
            'all_nonself_scores': all_nonself_scores
        }

### Test the model

In [ ]:
""" 
Let's load the model
- Set the default_device to specify where all the non-expert layers live (the experts are moved on model init)
- Set the default_dtype to specify the model dtype, all params will be in this dtype except for this explicitly specified differently in class definition
  - In the default OlMoE, RMSNorm is required to be f32 whereas all other params are bf16. 
"""
# torch.set_default_device(main_device) # This is buggy, don't use
torch.set_default_dtype(torch.bfloat16)
torch.set_float32_matmul_precision('medium') # See https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html 
torch.manual_seed(seed)
torch._dynamo.config.capture_scalar_outputs = True

model = OlmoeModel(
    conf,
    primary_device = main_device, # Where to store dense layers and shared experts
    expert_device_map = ['cuda:0'] * 48 # We have 4 experts, here let's test them with all of them on cuda:0
)
model = torch.compile(model)
total_params = sum(p.numel() for p in model.parameters()) 
active_params = total_params - sum(p.numel() * (1 - conf.top_k/conf.n_experts) for p in model.layers[0].moe.experts.parameters()) * conf.n_layers
print(f"Active/total parameters: {active_params:,}/{total_params:,}")
check_memory()

In [ ]:
""" 
Let's load a forward pass with a batch size of 2, to make sure the model is able to run
- If you have multiple working forward methods, this is a good chance to test them for equality
"""
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/OLMoE-1B-7B-0924', add_eos_token = False, add_bos_token = False)
prompt = ['I am a dog and I like to eat. My favorite food is', 'My cat is']
inputs = tokenizer(prompt, truncation = True, max_length = 128, padding = 'max_length', return_tensors = 'pt').to(main_device)

with torch.no_grad():
    output = model(inputs['input_ids'], inputs['attention_mask'], moe_method = 'forward_slow', use_lflb = True, use_checkpointing = False)
    # output = model(inputs['input_ids'], inputs['attention_mask'], moe_method = 'forward_fast', use_lflb = True, use_checkpointing = False)
    # output = model(inputs['input_ids'], inputs['attention_mask'], moe_method = 'forward_async', use_lflb = True, use_checkpointing = False)

output_ids = torch.argmax(output['logits'][:, :, :], dim = 2)
for i in range(output_ids.size(0)):
    idx = inputs["attention_mask"].sum(dim = -1)[i].item() - 1 # get length of attention mask to find the last non-mask output token ix
    print(tokenizer.decode(output_ids[i, idx], skip_special_tokens=True))

In [ ]:
# It's B x H x N
output['all_nonself_scores'][8] # First get mean across heads and batch

In [ ]:
with torch.no_grad():
    print(profile_memory(
        model,
        runs = 20,
        input_ids = inputs['input_ids'],
        attention_mask = inputs['attention_mask'],
        moe_method = 'forward_slow',
        use_lflb = False,
        use_checkpointing = False
    ))

# {'runs': 20, 'average_time': '0.03535322s', 'average_peak_mem': '606.6673MB', 'average_increase_mem_MB': '1.3112MB'} # compiled
# {'runs': 20, 'average_time': '0.03313379s', 'average_peak_mem': '761.1628MB', 'average_increase_mem_MB': '1.3112MB'} # non-compiled

### Test Interp Logging

In [ ]:
from helpers.expert_specialization import get_context_labeled_data, get_tis, get_ics, get_ec_all_tokens, get_ec

context_aware_test_dataset = get_context_labeled_data("./../../data/contextual-tokens/samples_*.yaml", tokenizer, 512, 16)

In [ ]:
@torch.no_grad()
def get_cos_sim(model, model_conf, main_device):
    """ 
    Custom get_cos_sim due to the different gating nesting in RRS class - this gets cos sim of the gate OFFSET
    """
    def check_cosine_similarity(weights):
        """
        weights: weight matrix on one router: [n_experts, D]
        """    
        weights_normalized = torch.nn.functional.normalize(weights, p=2, dim=1).to(dtype=torch.float32)
        cos_sim = torch.matmul(weights_normalized, weights_normalized.t())
        return cos_sim

    # Use model.shared_gate.weight (for shared gate) or model.layers[i].moe.gate_offset.weight (for offset)
    sim_matrices = [check_cosine_similarity(model.shared_gate.weight) for i in range(model_conf.n_layers)]
    identity_matrix = torch.eye(model_conf.n_experts).to(main_device)
    distances = [
        torch.norm(sim_matrix - identity_matrix, p = 'fro') / model_conf.n_experts * math.sqrt(model_conf.Z)  # Normalize by number of experts and sqrt of hidden dimension
        for sim_matrix in sim_matrices
        ]
    
    mean_distance = torch.mean(torch.stack(distances)).detach().cpu().item()
    return {
        'mean_distance': mean_distance, 
        'distance_by_layer': [x.detach().cpu().item() for x in distances]
    }

get_cos_sim(model, conf, main_device)

In [ ]:
# print(get_ics(model, context_aware_test_dataset[0:15], main_device, use_topk1 = False))
# print(get_tis(model, context_aware_test_dataset[0:20], main_device, tokenizer.pad_token_id, use_topk1 = False))
# print(get_cos_sim(model, conf, main_device))
# print(get_ec(model, context_aware_test_dataset[0:15], main_device, use_topk1 = False))
# print(get_ics(model, context_aware_test_dataset[0:15], main_device, use_topk1 = True))
# print(get_tis(model, context_aware_test_dataset[0:20], main_device, tokenizer.pad_token_id, use_topk1 = True))

### RRS Metrics

In [ ]:
@torch.no_grad()
def get_rrs_norms(all_rrs_states: list[torch.Tensor], attention_mask: torch.Tensor):
    """
    Given a list of RRS states [(B, N, Z), ... ] across layers, return some aggregated stats for each layer.
    """
    stats = {}
    mask_flat = (attention_mask.view(-1) == 1)

    for layer_ix, rrs in enumerate(all_rrs_states):
        # Reshape rrs to (BN, Z)
        rrs_flat = rrs.view(-1, rrs.size(-1))
        valid_rrs = rrs_flat[mask_flat]
        mean_val = valid_rrs.mean().item()
        std_val = valid_rrs.std().item()
        norm_val = valid_rrs.norm(p = 2).item()  # L2 norm
        stats[f'layer_{layer_ix}'] = {'mean': mean_val, 'std': std_val, 'norm': norm_val}
    return stats

get_rrs_norms(output['all_rrs_states'], inputs['attention_mask'])

### Test training

In [ ]:
"""
First, load some validation data as a dataloader. This will be needed for evaluation during model training later. 
- The function `load_shard_as_dataloader` loads JSON data shards, concatenates them with a seperator in between, then splits them by a given seq length.
- We do not load the dataloader for training data yet; the total size is too large, so instead these will be loaded later as needed in the training loop.
""" 
from helpers.dataset import load_shard_as_dataloader, load_pt_shard_as_dataloader

val_dl = load_shard_as_dataloader(
    './../../data/val_shard.json',
    tokenizer,
    batch_size = 32,
    seq_len = 2048,
    eos_seperator_id = tokenizer.eos_token_id,
    shuffle = False
)

In [ ]:
""" 
Now, define a function for calculating validation metrics. This will be later used in the training loop.
- An untrained model should typically return validation loss of ~10 for the base cross-entropy loss.
"""
@torch.no_grad()
def get_val_stats(model, val_dl, router_aux_loss_coef):
    """
    Get eval set metrics
    """
    model.eval()

    val_loss_sum = 0.0
    val_base_sum = 0.0
    val_aux_sum = 0.0
    val_steps = 0

    for val_batch in val_dl:
        val_input_ids = val_batch['input_ids'].to(main_device)
        val_attn_mask = val_batch['attention_mask'].to(main_device)
        
        test_outputs = model(val_input_ids, val_attn_mask, moe_method = 'forward_slow', use_checkpointing = False)

        val_base_sum += test_outputs['base_loss'].detach().cpu().item()
        val_aux_sum  += test_outputs['aux_loss'].detach().cpu().item()
        val_loss_sum += (test_outputs['base_loss'] + router_aux_loss_coef * test_outputs['aux_loss']).detach().cpu().item()
        
        val_steps += 1

    avg_test_loss = val_loss_sum / val_steps
    avg_test_base = val_base_sum / val_steps
    avg_test_aux  = val_aux_sum  / val_steps

    model.train()
    return {
        "loss": avg_test_loss,
        "base_loss": avg_test_base,
        "aux_loss": avg_test_aux
    }

get_val_stats(model, val_dl, 1e-2)

In [ ]:
@torch.no_grad()
def track_expert_usage(model, val_dl, out_pt_path: str = "expert_usage_long.pt", main_device: str = "cuda"):
    """
    Runs through the validation set, retrieves model outputs (including top-k experts),
    and saves them in a "long" format to a .pt file.

    The saved format is a Python dict with lists of equal length, containing:
      - 'batch':       Batch index (int)
      - 'token_idx':   Index within the sequence (int)
      - 'token_id':    Raw token ID (int)
      - 'layer_idx':   Layer index (int)
      - 'topk_slot':   Which of the top-k slots (int, range [0..top_k-1])
      - 'expert_id':   Expert ID chosen for that slot (int)

    Params:
        @model (nn.Module): Your OlmoeModel instance.
        @val_dl (DataLoader): Validation dataloader, yielding dicts with "input_ids", "attention_mask", etc. - DO NOT SHUFFLE!
        @out_pt_path (str): Where to write the .pt file of usage data.
        @main_device (str): Device name, e.g. "cuda" or "cpu".

    # loaded_data = torch.load(f's_{str(0)}.pt')
    # print(loaded_data.keys())  
    # # dict_keys(['batch', 'token_idx', 'token_id', 'layer_idx', 'topk_slot', 'expert_id'])

    # len_of_data = len(loaded_data["batch"])
    # print("Number of rows:", len_of_data)


    # import pandas as pd

    # df = pd.DataFrame(loaded_data)
    # df.head()
    """

    # We'll store usage info in lists. Later we can save them as a .pt file (dict of lists).
    all_records = {
        "batch_ix": [],
        "seq_ix": [],
        "token_ix": [],
        "token_id": [],
        "layer_ix": [],
        "topk_slot": [],
        "expert_id": [],
    }

    for batch_ix, val_batch in enumerate(val_dl):

        val_input_ids = val_batch["input_ids"].to(main_device)
        val_attn_mask = val_batch["attention_mask"].to(main_device)
        outputs = model(input_ids = val_input_ids, attention_mask = val_attn_mask, moe_method = 'forward_slow', use_checkpointing= False)


        # 'all_topk_experts' is a tuple of length n_layers each entry is shape [B*N, top_k]
        all_topk_experts = outputs["all_topk_experts"]
        n_layers = len(all_topk_experts)

        B, N = val_input_ids.shape

        # Iterate over tokens in the batch
        for b in range(B):
            for n_idx in range(N):
                # If attention_mask=0, it's typically padding => skip
                if val_attn_mask[b, n_idx].item() == 0:
                    continue

                token_id = val_input_ids[b, n_idx].item()
                offset = b * N + n_idx

                # For each layer, retrieve the top-k experts
                for layer_idx in range(n_layers):
                    topk_exps_tensor = all_topk_experts[layer_idx][offset]  # shape [top_k]
                    
                    # For each top-k slot, record a row in "long" format
                    for slot_idx, ex_id in enumerate(topk_exps_tensor):
                        all_records["batch_ix"].append(batch_ix)
                        all_records['seq_ix'].append(b)
                        all_records["token_ix"].append(n_idx)
                        all_records["token_id"].append(token_id)
                        all_records["layer_ix"].append(layer_idx)
                        all_records["topk_slot"].append(slot_idx)
                        all_records["expert_id"].append(ex_id.item())

        if batch_ix >= 1:
            break

    # Save to .pt file
    torch.save(all_records, out_pt_path)

    print(f"Saved long-format expert usage to {out_pt_path}.")
    print(f"Total records: {len(all_records['batch_ix'])}")

track_expert_usage(model, val_dl, out_pt_path = 'logs/test.pt')

In [ ]:
""" 
Set training constants to be used for training later.
- The batch size will be equal to micro_batch_size * accumulation_steps.
"""
@dataclass
class TrainConf:
    router_aux_loss_coef: float = 0.001  # Base OlMoE: 0.01 (relative weight of balancing loss)
    use_lflb: bool = True # Use loss-free load balancing
    bias_update_rate: float = .001 # Bias update rate for lflb
    lr: float = 5e-4 # The starting LR (after warmup)
    min_lr: float = 5e-5 # The minimum LR
    warmup_steps: int = 100 # How long it takes to warmup to the starting LR
    decay_steps: int = 49900 # How long it takes to decay from the starting LR to the minimum LR
    max_grad_norm: float = 1.0 # Gradient clipping for non-expert grads
    max_expert_grad_norm: float = 1.0 # Gradient clipping for expert grads
    micro_batch_size: int = 64 # Size of a microbatch
    accumulation_steps: int = 4 # Number of microbatches within a batch
    seq_len: int = 2048 # The sequence length
    rrs_offset_loss_coef: int = 1e-1

train_conf = TrainConf()

In [ ]:
"""
Setup a Wandb run for logging. Choose a run name and notes for the run!
"""
RUN_NAME = 'test-50 RRS test with RRS-attention!'
RUN_NOTES = 'RRS test with learnable RRS attention and Z = 32'

load_dotenv('./../../secrets.env')
wandb.login(key = os.getenv('WANDB_API_KEY'))
run = wandb.init(
    project = 'interpretable-moes', 
    name = RUN_NAME,
    notes = RUN_NOTES,
    config = {**asdict(conf), **asdict(train_conf)}
)

# (Optional) Also log various info as a wandb media object.
additional_log_notes = {
    'run_name': RUN_NAME,
    'notes': RUN_NOTES,
    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_params':total_params,
    'active_params': active_params,
    'available_cuda_gpus': [torch.cuda.get_device_properties(i).name for i in range(torch.cuda.device_count())],
    'model_conf': asdict(conf),
    'train_conf': asdict(train_conf)
}

wandb.log({'conf': wandb.Html(f"<pre style='font-size:12px;'>{json.dumps(additional_log_notes, indent = 2)}</pre>")})

In [ ]:
"""
Aux loss coef
"""
def get_linear_aux_coef(step: int, max_steps: int, start_coef: float, end_coef: float) -> float:
    if step >= max_steps:
        return end_coef
    frac = step / max_steps
    return start_coef + frac * (end_coef - start_coef)

get_linear_aux_coef(0, train_conf.decay_steps + train_conf.warmup_steps, start_coef = train_conf.router_aux_loss_coef, end_coef = 0)

In [ ]:
"""
Let's train the model.
- The training loop will loop through training data shards. Each shard will be loaded and concatenated into chunks of size seq_len.
- Things to consider implementing in the future: more aggressive router LR decay (to encourage router stability)
"""
# Initialize optimizer/scheduler. The scheduler combines a warmup + cosine annealing.
optimizer = torch.optim.AdamW(model.parameters(), lr = train_conf.lr, fused = True)
scheduler = torch.optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers = [
        torch.optim.lr_scheduler.LinearLR(optimizer, start_factor = 0.2, end_factor = 1.0, total_iters = train_conf.warmup_steps), # Warmup
        torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_conf.decay_steps, eta_min = train_conf.min_lr, last_epoch = -1) # Cosine annealing
    ],
    milestones = [train_conf.warmup_steps]
)

# Look for all training data files
shard_files = sorted(glob.glob("./../../data/olmoe-tokenizer/train_shard_*.pt"))
print(f"Found {len(shard_files)} shards.")

# Initialize step count
step = 0
total_tokens_trained = 0
model.train()
torch.manual_seed(seed)

for shard_idx, shard_path in enumerate(shard_files):

    print(f"\n=== Loading shard {shard_path} (index {shard_idx}) ===")
    shard_dl = load_pt_shard_as_dataloader(shard_path, tokenizer, batch_size = train_conf.micro_batch_size * train_conf.accumulation_steps, seq_len = train_conf.seq_len, shuffle = True)

    for batch_idx, batch in enumerate(shard_dl):

        # ====================== SPLIT BATCH INTO MICRO-BATCHES ======================
        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)

        if input_ids.size(0) < (train_conf.accumulation_steps * train_conf.micro_batch_size):
            print(f"Skipping leftover batch, need at least {train_conf.accumulation_steps * train_conf.micro_batch_size}")
            continue

        sub_input_ids = input_ids.split(train_conf.micro_batch_size, dim = 0) 
        sub_attn_mask = attention_mask.split(train_conf.micro_batch_size, dim = 0)

        # ====================== ZERO GRAD ONCE PER "BIG BATCH" ======================
        optimizer.zero_grad()
        
        # We'll track times and losses across micro-batches
        total_fwd_time = 0.0
        total_bwd_time = 0.0
        total_loss_val = 0.0
        start_batch = time.time()

        # We'll keep a list of dictionaries, one per layer, each mapping expert_id -> usage_count
        usage_accum = [defaultdict(int) for _ in range(model.conf.n_layers)]

        # ====================== MICRO-BATCH LOOP ======================
        for i in range(train_conf.accumulation_steps):

            mb_input_ids = sub_input_ids[i]
            mb_attn_mask = sub_attn_mask[i]

            # ---------------------- Forward ----------------------
            start_fwd = time.time()
            outputs = model(mb_input_ids, mb_attn_mask, moe_method = 'forward_slow', use_lflb = train_conf.use_lflb, use_checkpointing = True)
            aux_coef = get_linear_aux_coef(step, train_conf.decay_steps + train_conf.warmup_steps, start_coef = train_conf.router_aux_loss_coef, end_coef = 0)
            loss = outputs['base_loss'] + aux_coef * outputs['aux_loss'] + train_conf.rrs_offset_loss_coef * outputs['rrs_offset_loss']
            fwd_time = time.time() - start_fwd
            total_fwd_time += fwd_time

            # ---------------------- Collect Expert Usage for This Micro-Batch ----------------------
            with torch.no_grad():
                all_topk_experts = outputs['all_topk_experts']
                for layer_idx, topk_expert_tensor in enumerate(outputs["all_topk_experts"]):
                    flat_experts = topk_expert_tensor.view(-1)  
                    unique_experts = flat_experts.unique()
                    for ex_id in unique_experts:
                        ex_count = (flat_experts == ex_id).sum().item()
                        usage_accum[layer_idx][int(ex_id)] += ex_count
                        
            # ---------------------- Backward ----------------------
            # Divide by accumulation_steps so total gradient matches "big batch" size
            scaled_loss = loss / train_conf.accumulation_steps
            start_bwd = time.time()
            scaled_loss.backward()
            bwd_time = time.time() - start_bwd
            total_bwd_time += bwd_time

            total_loss_val += loss.item()

        # ====================== GRAD CLIPPING & OPT STEP ======================
        shared_params = [p for n,p in model.named_parameters() if 'expert' not in n]
        expert_params = [p for n,p in model.named_parameters() if 'expert' in n]

        torch.nn.utils.clip_grad_norm_(shared_params, train_conf.max_grad_norm)
        torch.nn.utils.clip_grad_norm_(expert_params, train_conf.max_expert_grad_norm)

        optimizer.step()
        scheduler.step()

        # ====================== LOSS-FREE BIAS UPDATE ======================
        # We'll do sign-based bias updates after each "big batch"
        if train_conf.use_lflb:
            for layer_ix in range(model.conf.n_layers):
                model.layers[layer_ix].moe.update_expert_biases(usage_accum[layer_ix], train_conf.bias_update_rate)

        # ============== METRICS ==============
        avg_loss = total_loss_val / train_conf.accumulation_steps # Take the average loss over micro-batches. total_loss_val is the sum of 'loss.item()'.
        total_tokens_trained += attention_mask.sum().detach().cpu().item()
        metrics = {
            'step': step,
            'shard_idx': shard_idx,
            'batch_size': input_ids.shape[0],
            'total_tokens_trained': total_tokens_trained,
            'lr': optimizer.param_groups[0]['lr'],
            'aux_coef': aux_coef,
            'train': {
                'loss': avg_loss,
                'base_loss': outputs['base_loss'].detach().cpu().item(), # From last microbatch only
                'aux_loss':  outputs['aux_loss'].detach().cpu().item(), # From last microbatch only
                'rrs_offset_loss': outputs['rrs_offset_loss'].detach().cpu().item(), # From last microbatch only
            },
            'fwd_time':  total_fwd_time,
            'bwd_time':  total_bwd_time,
            'batch_time':  time.time() - start_batch
        }

        # ============== EXPENSIVE METRICS (EVERY 10 STEPS) ==============
        if step % 10 == 0:
            metrics['gradients'] = get_gradient_stats(model)
            # Convert usage_accum (list of defaultdicts) into a more standard dict for logging
            usage_dict_final = {}
            for layer_idx, ex_dict in enumerate(usage_accum):
                usage_dict_final[layer_idx] = dict(ex_dict)  # convert defaultdict -> normal dict
            metrics['expert_usage'] = usage_dict_final
            metrics['alpha'] = dict(enumerate([layer._alpha_param.detach().item() for layer in model.layers]))
            metrics['alpha_sigmoid'] = dict(enumerate([0.5 + 0.5 * torch.tanh(layer._alpha_param.detach()).item() for layer in model.layers]))
            metrics['gamma'] = dict(enumerate([layer._gamma_param.detach().item() for layer in model.layers]))
            metrics['gamma_sigmoid'] = dict(enumerate([0.5 + 0.5 * torch.tanh(layer._gamma_param.detach()).item() for layer in model.layers]))
            metrics['all_nonself_scores'] = dict(enumerate([x.detach().item() for x in outputs['all_nonself_scores']]))

        # ============== SOMEWHAT EXPENSIVE METRICS (EVERY 200 STEPS) ==============
        if (step <= 200 and step % 50 == 0) or (step <= 2000 and step % 100 == 0) or (step % 200 == 0):
            metrics['val'] = get_val_stats(model, val_dl, train_conf.router_aux_loss_coef)
            metrics['ec'] = get_ec_all_tokens(outputs['all_topk_experts'], mb_attn_mask, model.conf.n_experts, 4)
            metrics['rrs_norms'] = get_rrs_norms(outputs['all_rrs_states'], mb_attn_mask)

        # ============== EXTRA EXPENSIVE METRICS (EVERY 500 STEPS) ==============
        if (step <= 1000 and step % 100 == 0) or (step <= 2000 and step % 200 == 0) or (step % 500 == 0):
            metrics['ics'] = get_ics(model, context_aware_test_dataset[0:15], main_device, use_topk1 = False)
            metrics['tis'] = get_tis(model, context_aware_test_dataset[0:15], main_device, tokenizer.pad_token_id, use_topk1 = False)
            metrics['ec_by_token'] = get_ec(model, context_aware_test_dataset[0:15], main_device, use_topk1 = False)
            metrics['ics1'] = get_ics(model, context_aware_test_dataset[0:15], main_device, use_topk1 = True)
            metrics['tis1'] = get_tis(model, context_aware_test_dataset[0:15], main_device, tokenizer.pad_token_id, use_topk1 = True)
            metrics['cos_sim'] = get_cos_sim(model, conf, main_device)
            
        if (step > 0 and step <= 1000 and step % 100 == 0) or (step <= 2000 and step % 200 == 0) or (step % 500 == 0):
            track_expert_usage(model, val_dl, f'logs/s_{str(total_tokens_trained).rjust(15, '0')}.pt', main_device)

        # ============== SAVE (EVERY 5000 STEPS) ==============
        if step > 0 and step % 2500 == 0:
            os.makedirs(os.path.dirname('./saves'), exist_ok = True)
            torch.save(
                {
                    'model_class': dill.dumps(model.__class__),
                    'model_state_dict': model.state_dict(), 
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'total_tokens_trained': total_tokens_trained,
                    'step': step,
                },
                f"saves/checkpoint_{step:08d}.pt",
                pickle_module = dill
                )

        # ============== LOG TO W&B ==============
        wandb.log(metrics)

        # ============== PRINT ==============
        if step <= 10 or (step <= 100 and step % 10 == 0) or (step > 100 and step % 100 == 0):
            print(f"Step {step}: avg_loss={metrics['train']['loss']:.4f} "
                f"| fwd_time={metrics['fwd_time']:.2f}s | bwd_time={metrics['bwd_time']:.2f}s | batch_time = {metrics['batch_time']:.2f} "
                f"| lr={metrics['lr']:.1e}"
            )
            
        step += 1

wandb.finish()

In [ ]:
""" 
Final save
""" 
from helpers.memory import clear_all_cuda_memory

torch.save(
    {
        'model_class': dill.dumps(model.__class__),
        'model_state_dict': model.state_dict(), 
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'total_tokens_trained': total_tokens_trained,
        'step': step,
    },
    f"saves/checkpoint_{step:08d}.pt",
    pickle_module = dill
    )

clear_all_cuda_memory()
check_memory()

In [ ]:
"""
Qualitative test
"""
prompt = 'My dog likes to eat '

inputs = tokenizer(prompt, return_tensors = 'pt').to(main_device)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
tokenizer = AutoTokenizer.from_pretrained('allenai/OLMoE-1B-7B-0924', add_eos_token = False, add_bos_token = False)

# Iteratively generate tokens
with torch.no_grad():
    for _ in range(255):
        output = model(input_ids, attention_mask, moe_method = 'forward_slow', use_checkpointing = False)['logits']

        next_token_id = torch.argmax(output[0, -1, :], dim = -1).unsqueeze(0)

        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim = 1)
        attention_mask = torch.cat([attention_mask, torch.ones((1, 1), dtype = torch.long, device = input_ids.device)], dim = 1)

        if next_token_id.item() in [tokenizer.eos_token_id, tokenizer.encode('\n')[0]]:
            break

# Decode final sequence
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens = False)
print(generated_text)